In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/PDI')

In [ ]:
!wget --no-check-certificate \
    https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip \
    -O /tmp/ISIC2018_Task1-2_Test_Input.zip

--2023-08-10 23:25:13--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 54.231.131.225, 3.5.25.122, 52.217.132.241, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)|54.231.131.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2370457338 (2.2G) [application/zip]
Saving to: ‘/tmp/ISIC2018_Task1-2_Test_Input.zip’

/tmp/ISIC2018_Task1 100%[===================>]   2.21G  68.6MB/s    in 35s     

2023-08-10 23:25:49 (64.2 MB/s) - ‘/tmp/ISIC2018_Task1-2_Test_Input.zip’ saved [2370457338/2370457338]



In [ ]:
!wget --no-check-certificate \
     https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Test_GroundTruth.zip \
    -O /tmp/ISIC2018_Task1_Test_GroundTruth.zip

--2023-08-10 23:25:49--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Test_GroundTruth.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 52.217.166.177, 16.182.41.169, 52.217.104.100, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)|52.217.166.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9680101 (9.2M) [application/zip]
Saving to: ‘/tmp/ISIC2018_Task1_Test_GroundTruth.zip’

/tmp/ISIC2018_Task1 100%[===================>]   9.23M  31.7MB/s    in 0.3s    

2023-08-10 23:25:49 (31.7 MB/s) - ‘/tmp/ISIC2018_Task1_Test_GroundTruth.zip’ saved [9680101/9680101]



In [ ]:
import zipfile

local_zip = '/tmp/ISIC2018_Task1-2_Test_Input.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
import zipfile

local_zip = '/tmp/ISIC2018_Task1_Test_GroundTruth.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.optimizers import Adam
from keras.metrics import Recall, Precision
from model import build_unet
from metrics import dice_coef, iou

H = 256
W = 256

"""
A função create_dir(path) tem como objetivo criar um novo diretório (pasta) em um determinado caminho (path)
especificado pelo usuário, caso esse diretório ainda não exista.
"""


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


"""
A função shuffling(x, y) tem como objetivo embaralhar os dados de entrada x e y usando a função shuffle da biblioteca
sklearn.utils. Essa função é frequentemente usada em problemas de aprendizado de máquina para evitar que o modelo seja
viesado por uma ordem específica nos dados de treinamento.
"""


def shuffling(img, mask):
    img, mask = shuffle(img, mask, random_state=42)
    return img, mask


"""
A função load_data(data_path, split=0.2) tem como objetivo carregar e dividir o conjunto de dados de imagens para
treinamento, validação e testes. São usadas algumas função para conseguir separar as imagens e suas respectivas masks,
para que possam ser realizados os treinamentos, validações e tests. A tupla retornada irá ser usada para alimentar  nosso
modelo.
"""

def load_data(data_path, split=0.4):
    images_folder = '/tmp/ISIC2018_Task1-2_Test_Input'
    masks_folder = '/tmp/ISIC2018_Task1_Test_GroundTruth'

    # Não é necessário usar glob para listar diretórios, apenas para listar arquivos neles
    images = [os.path.join(images_folder, img) for img in os.listdir(images_folder) if img.endswith('.jpg')]
    masks = [os.path.join(masks_folder, mask) for mask in os.listdir(masks_folder) if mask.endswith('.png')]

    print("Número de imagens:", len(images))
    print("Número de máscaras:", len(masks))

    test_size = int(len(images) * split)

    train_img, valid_img = train_test_split(images, test_size=test_size, random_state=42)
    train_mask, valid_mask = train_test_split(masks, test_size=test_size, random_state=42)

    train_img, test_img = train_test_split(train_img, test_size=test_size, random_state=42)
    train_mask, test_mask = train_test_split(train_mask, test_size=test_size, random_state=42)

    return (train_img, train_mask), (valid_img, valid_mask), (test_img, test_mask)


"""
A função read_image(path) tem como objetivo ler a imagem colorida e retornar uma representação da imagem em formato
adequado para o processamento em nosso modelo.
"""


def read_image(path):
    path = path.decode()
    img = cv2.imread(path, cv2.IMREAD_COLOR)  # (H, W, 3)
    img = cv2.resize(img, (W, H))
    img = img / 255.0
    img = img.astype(np.float32)
    return img  # (256, 256, 3)


"""
A função read_mask(path) tem como objetivo ler a mascara em escala de cinza e retornar uma representação da mascara
em formato adequado para o processamento em nosso modelo.
"""


def read_mask(path):
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # (H, W)
    mask = cv2.resize(mask, (W, H))
    mask = mask / 255.0
    mask = mask.astype(np.float32)  # (256, 256)
    mask = np.expand_dims(mask, axis=-1)  # (256, 256, 1)
    return mask


"""
A função tf_parse(x, y) é uma função de pré-processamento de dados que é usada para carregar e pré-processar
imagens e suas máscaras correspondentes. A função é definida como uma função aninhada `_parse`, que carrega as
imagens e máscaras usando as funções `read_image` e `read_mask`. Em seguida, a função `_parse` é envolvida em um
`tf.numpy_function`, que permite que a função Python seja usada como uma operação TensorFlow. A função
tf.numpy_function retorna uma lista de dois tensores: `x` e `y`, que têm dimensões dinâmicas. Portanto, é necessário
definir as dimensões dos tensores usando o método `set_shape()`. Por fim, a função `tf_parse` retorna os tensores `x`
e `y` com dimensões definidas, que são usados para alimentar o modelo de aprendizado de máquina.
"""


def tf_parse(img, mask):
    def _parse(img, mask):
        img = read_image(img)
        mask = read_mask(mask)
        return img, mask

    img, mask = tf.numpy_function(_parse, [img, mask], [tf.float32, tf.float32])
    img.set_shape([H, W, 3])
    mask.set_shape([H, W, 1])
    return img, mask


"""
A função tf_dataset(X, Y, batch) é usada para criar um objeto de conjunto de dados TensorFlow a partir de um
conjunto de dados de entrada, que é composto por uma lista de caminhos de imagens e uma lista de caminhos de máscaras
correspondentes. A função pré-processa os dados usando a função tf_parse, que carrega e pré-processa as imagens e
máscaras correspondentes. Em seguida, o conjunto de dados é dividido em lotes e configurado para carregar exemplos de
lote de forma assíncrona em segundo plano para melhorar o desempenho durante o treinamento. Por fim, a função retorna
o objeto de conjunto de dados processado, que é usado para alimentar o modelo de aprendizado de máquina.
"""


def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 4
    lr = 1e-4  ## (0.0001)
    num_epoch = 3
    model_path = "files/split-0.4-1003-1003-imgs-masks-5-epoch/model.h5"
    csv_path = "files/split-0.4-1003-1003-imgs-masks-5-epoch/data.csv"

    """ Dataset : 60/20/20 """
    data_path = "ISIC2018-Challenge/"
    (train_img, train_mask), (valid_img, valid_mask), (test_img, test_mask) = load_data(data_path)

    print(f"Train: {len(train_img)} - {len(train_mask)}")
    print(f"Valid: {len(valid_img)} - {len(valid_mask)}")
    print(f"Test: {len(test_img)} - {len(test_mask)}")

    train_dataset = tf_dataset(train_img, train_mask, batch_size)
    valid_dataset = tf_dataset(valid_img, valid_mask, batch_size)

    train_steps = len(train_img) // batch_size
    valid_steps = len(valid_img) // batch_size

    if len(train_img) % batch_size != 0:
        train_steps += 1

    if len(valid_img) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = build_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
    model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Número de imagens: 1000
Número de máscaras: 1000
Train: 200 - 200
Valid: 400 - 400
Test: 400 - 400
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 256, 256, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 256, 256, 64  256        ['conv2d_38[0][0]']        